# Session 5 – Multi-Agent Orchestrator

Ipinapakita ang isang simpleng pipeline na may dalawang ahente (Mananaliksik -> Editor) gamit ang Foundry Local.


### Paliwanag: Pag-install ng Dependency
Ini-install ang `foundry-local-sdk` at `openai` na kinakailangan para sa lokal na pag-access ng modelo at mga chat completions. Idempotent.


# Scenario
Nagpapatupad ng minimal na pattern ng orchestrator na may dalawang ahente:
- **Researcher agent** nangongolekta ng maikli at tiyak na mga impormasyon
- **Editor agent** muling isinusulat para sa malinaw na presentasyon sa mga executive

Ipinapakita ang shared memory bawat ahente, sunod-sunod na pagpapasa ng intermediate output, at isang simpleng pipeline function. Maaaring palawakin para sa mas maraming tungkulin (hal., Critic, Verifier) o parallel na mga sangay.

**Mga Environment Variable:**
- `FOUNDRY_LOCAL_ALIAS` - Default na modelong gagamitin (default: phi-4-mini)
- `AGENT_MODEL_PRIMARY` - Pangunahing modelo ng ahente (ina-override ang ALIAS)
- `AGENT_MODEL_EDITOR` - Modelo ng Editor agent (default sa pangunahing modelo)

**SDK Reference:** https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local

**Paano ito gumagana:**
1. **FoundryLocalManager** awtomatikong sinisimulan ang Foundry Local service
2. Dina-download at ina-load ang tinukoy na modelo (o ginagamit ang naka-cache na bersyon)
3. Nagbibigay ng OpenAI-compatible na endpoint para sa interaksyon
4. Bawat ahente ay maaaring gumamit ng ibang modelo para sa espesyalisadong mga gawain
5. Ang built-in na retry logic ay maayos na humahawak sa mga pansamantalang pagkabigo

**Mga Pangunahing Tampok:**
- ✅ Awtomatikong pagtuklas at pagsisimula ng serbisyo
- ✅ Pamamahala ng lifecycle ng modelo (download, cache, load)
- ✅ OpenAI SDK compatibility para sa pamilyar na API
- ✅ Suporta sa multi-model para sa espesyalisasyon ng ahente
- ✅ Matibay na error handling gamit ang retry logic
- ✅ Lokal na inference (hindi kinakailangan ang cloud API)


In [16]:
# Install dependencies
!pip install -q foundry-local-sdk openai

### Paliwanag: Pangunahing Imports at Typing
Nagpapakilala ng dataclasses para sa pag-iimbak ng mga mensahe ng ahente at mga typing hint para sa kalinawan. Ina-import ang Foundry Local manager + OpenAI client para sa mga susunod na aksyon ng ahente.


In [17]:
from dataclasses import dataclass, field
from typing import List
import os
from foundry_local import FoundryLocalManager
from openai import OpenAI

### Paliwanag: Pag-inisyalisa ng Modelo (SDK Pattern)
Gumagamit ng Foundry Local Python SDK para sa maaasahang pamamahala ng modelo:
- **FoundryLocalManager(alias)** - Awtomatikong sinisimulan ang serbisyo at ina-upload ang modelo gamit ang alias
- **get_model_info(alias)** - Tinutukoy ang alias sa tiyak na model ID
- **manager.endpoint** - Nagbibigay ng service endpoint para sa OpenAI client
- **manager.api_key** - Nagbibigay ng API key (opsyonal para sa lokal na paggamit)
- Sinusuportahan ang magkakahiwalay na modelo para sa iba't ibang ahente (pangunahing ahente vs editor)
- May built-in na retry logic na may exponential backoff para sa katatagan
- Pag-verify ng koneksyon upang matiyak na handa ang serbisyo

**Pangunahing SDK Pattern:**
```python
manager = FoundryLocalManager(alias)
model_info = manager.get_model_info(alias)
client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key)
```

**Pamamahala ng Lifecycle:**
- Ang mga manager ay iniimbak nang globally para sa tamang pag-cleanup
- Ang bawat ahente ay maaaring gumamit ng ibang modelo para sa espesyalisasyon
- Awtomatikong pagtuklas ng serbisyo at paghawak ng koneksyon
- Maayos na pag-retry na may exponential backoff sa mga pagkabigo

Tinitiyak nito ang tamang inisyalisa bago magsimula ang orkestrasyon ng ahente.

**Sanggunian:** https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local


In [18]:
import time

# Environment configuration
PRIMARY_ALIAS = os.getenv('AGENT_MODEL_PRIMARY', os.getenv('FOUNDRY_LOCAL_ALIAS', 'phi-4-mini'))
EDITOR_ALIAS = os.getenv('AGENT_MODEL_EDITOR', PRIMARY_ALIAS)

# Store managers globally for proper lifecycle management
primary_manager = None
editor_manager = None

def init_model(alias: str, max_retries: int = 3):
    """Initialize Foundry Local manager with retry logic.
    
    Args:
        alias: Model alias to initialize
        max_retries: Number of retry attempts with exponential backoff
    
    Returns:
        Tuple of (manager, client, model_id, endpoint)
    """
    delay = 2.0
    last_err = None
    
    for attempt in range(1, max_retries + 1):
        try:
            print(f"[Init] Starting Foundry Local for '{alias}' (attempt {attempt}/{max_retries})...")
            
            # Initialize manager - this starts the service and loads the model
            manager = FoundryLocalManager(alias)
            
            # Get model info to retrieve the actual model ID
            model_info = manager.get_model_info(alias)
            model_id = model_info.id
            
            # Create OpenAI client with manager's endpoint
            client = OpenAI(
                base_url=manager.endpoint,
                api_key=manager.api_key or 'not-needed'
            )
            
            # Verify the connection with a simple test
            models = client.models.list()
            print(f"[OK] Initialized '{alias}' -> {model_id} at {manager.endpoint}")
            
            return manager, client, model_id, manager.endpoint
            
        except Exception as e:
            last_err = e
            if attempt < max_retries:
                print(f"[Retry {attempt}/{max_retries}] Failed to init '{alias}': {e}")
                print(f"[Retry] Waiting {delay:.1f}s before retry...")
                time.sleep(delay)
                delay *= 2
            else:
                print(f"[ERROR] Failed to initialize '{alias}' after {max_retries} attempts")
    
    raise RuntimeError(f"Failed to initialize '{alias}' after {max_retries} attempts: {last_err}")

# Initialize primary model (for researcher)
print(f"\n{'='*80}")
print(f"Initializing Primary Model: {PRIMARY_ALIAS}")
print('='*80)
primary_manager, primary_client, PRIMARY_MODEL_ID, primary_endpoint = init_model(PRIMARY_ALIAS)

# Initialize editor model (may be same as primary)
if EDITOR_ALIAS != PRIMARY_ALIAS:
    print(f"\n{'='*80}")
    print(f"Initializing Editor Model: {EDITOR_ALIAS}")
    print('='*80)
    editor_manager, editor_client, EDITOR_MODEL_ID, editor_endpoint = init_model(EDITOR_ALIAS)
else:
    print(f"\n[Info] Editor using same model as primary")
    editor_manager = primary_manager
    editor_client, EDITOR_MODEL_ID = primary_client, PRIMARY_MODEL_ID
    editor_endpoint = primary_endpoint

print(f"\n{'='*80}")
print(f"[Configuration Summary]")
print('='*80)
print(f"  Primary Agent:")
print(f"    - Alias: {PRIMARY_ALIAS}")
print(f"    - Model: {PRIMARY_MODEL_ID}")
print(f"    - Endpoint: {primary_endpoint}")
print(f"\n  Editor Agent:")
print(f"    - Alias: {EDITOR_ALIAS}")
print(f"    - Model: {EDITOR_MODEL_ID}")
print(f"    - Endpoint: {editor_endpoint}")
print('='*80)



Initializing Primary Model: phi-4-mini
[Init] Starting Foundry Local for 'phi-4-mini' (attempt 1/3)...
[OK] Initialized 'phi-4-mini' -> Phi-4-mini-instruct-cuda-gpu:4 at http://127.0.0.1:59959/v1

Initializing Editor Model: gpt-oss-20b
[Init] Starting Foundry Local for 'gpt-oss-20b' (attempt 1/3)...
[OK] Initialized 'gpt-oss-20b' -> gpt-oss-20b-cuda-gpu:1 at http://127.0.0.1:59959/v1

[Configuration Summary]
  Primary Agent:
    - Alias: phi-4-mini
    - Model: Phi-4-mini-instruct-cuda-gpu:4
    - Endpoint: http://127.0.0.1:59959/v1

  Editor Agent:
    - Alias: gpt-oss-20b
    - Model: gpt-oss-20b-cuda-gpu:1
    - Endpoint: http://127.0.0.1:59959/v1


### Paliwanag: Mga Klase ng Agent at Memory
Nagpapakilala ng magaan na `AgentMsg` para sa mga entry ng memorya at `Agent` na sumasaklaw sa:
- **Sistema ng papel** - Persona ng agent at mga tagubilin
- **Kasaysayan ng mensahe** - Pinapanatili ang konteksto ng pag-uusap
- **act() method** - Isinasagawa ang mga aksyon na may tamang paghawak sa error

Maaaring gumamit ang agent ng iba't ibang modelo (pangunahing vs editor) at pinapanatili ang hiwalay na konteksto bawat agent. Ang pattern na ito ay nagbibigay-daan sa:
- Pagpapanatili ng memorya sa bawat aksyon
- Flexible na pagtatalaga ng modelo bawat agent
- Paghiwalay ng error at pag-recover
- Madaling pag-chain at pag-orchestrate


In [19]:
@dataclass
class AgentMsg:
    role: str
    content: str

@dataclass
class Agent:
    name: str
    system: str
    client: OpenAI = None  # Allow per-agent client assignment
    model_id: str = None   # Allow per-agent model
    memory: List[AgentMsg] = field(default_factory=list)

    def _history(self):
        """Return chat history in OpenAI messages format including system + memory."""
        msgs = [{'role': 'system', 'content': self.system}]
        for m in self.memory[-6:]:  # Keep last 6 messages to avoid context overflow
            msgs.append({'role': m.role, 'content': m.content})
        return msgs

    def act(self, prompt: str, temperature: float = 0.4, max_tokens: int = 300):
        """Send a prompt, store user + assistant messages in memory, and return assistant text.
        
        Args:
            prompt: User input/task for the agent
            temperature: Sampling temperature (0.0-1.0)
            max_tokens: Maximum tokens to generate
        
        Returns:
            Assistant response text
        """
        # Use agent-specific client/model or fall back to primary
        client_to_use = self.client or primary_client
        model_to_use = self.model_id or PRIMARY_MODEL_ID
        
        self.memory.append(AgentMsg('user', prompt))
        
        try:
            # Build messages including system prompt and history
            messages = self._history() + [{'role': 'user', 'content': prompt}]
            
            resp = client_to_use.chat.completions.create(
                model=model_to_use,
                messages=messages,
                max_tokens=max_tokens,
                temperature=temperature,
            )
            
            # Validate response
            if not resp.choices:
                raise RuntimeError("No completion choices returned")
            
            out = resp.choices[0].message.content or ""
            
            if not out:
                raise RuntimeError("Empty response content")
            
        except Exception as e:
            out = f"[ERROR:{self.name}] {type(e).__name__}: {str(e)}"
            print(f"[Agent Error] {self.name}: {type(e).__name__}: {str(e)}")
        
        self.memory.append(AgentMsg('assistant', out))
        return out

print("[INFO] Agent classes initialized with Foundry SDK support")
print(f"[INFO] Using OpenAI SDK version: {OpenAI.__module__}")


[INFO] Agent classes initialized with Foundry SDK support
[INFO] Using OpenAI SDK version: openai


### Paliwanag: Orchestrated Pipeline
Gumagawa ng dalawang espesyal na ahente:
- **Researcher**: Gumagamit ng pangunahing modelo, nangongolekta ng mga makatotohanang impormasyon
- **Editor**: Maaaring gumamit ng hiwalay na modelo (kung naka-configure), pinapaganda at muling isinusulat

Ang `pipeline` na function:
1. Kinokolekta ng Researcher ang hilaw na impormasyon
2. Pinapaganda ng Editor para maging handa sa executive-level na output
3. Ibinabalik ang parehong intermediate at final na resulta

Ang pattern na ito ay nagbibigay-daan sa:
- Espesyalisasyon ng modelo (magkaibang modelo para sa magkaibang tungkulin)
- Pagpapabuti ng kalidad sa pamamagitan ng multi-stage na pagproseso
- Pagsubaybay sa pagbabago ng impormasyon
- Madaling pagdaragdag ng mas maraming ahente o parallel na pagproseso


In [ ]:
# Create specialized agents with optional model assignment
researcher = Agent(
    name='Researcher',
    system='You collect concise factual bullet points.',
    client=primary_client,
    model_id=PRIMARY_MODEL_ID
)

editor = Agent(
    name='Editor',
    system='You rewrite content for clarity and an executive, action-focused tone.',
    client=editor_client,
    model_id=EDITOR_MODEL_ID
)

def pipeline(q: str, verbose: bool = True):
    """Execute multi-agent pipeline: Researcher -> Editor.
    
    Args:
        q: User question/task
        verbose: Print intermediate outputs
    
    Returns:
        Dictionary with research, final outputs, and metadata
    """
    if verbose:
        print(f"[Pipeline] Question: {q}\n")
    
    # Stage 1: Research
    if verbose:
        print("[Stage 1: Research]")
    research = researcher.act(q)
    if verbose:
        print(f"Output: {research[:200]}...\n")
    
    # Stage 2: Editorial refinement
    if verbose:
        print("[Stage 2: Editorial Refinement]")
    rewrite = editor.act(
        f"Rewrite professionally with a 1-sentence executive summary first. "
        f"Improve clarity, keep bullet structure if present. Source:\n{research}"
    )
    if verbose:
        print(f"Output: {rewrite[:200]}...\n")
    
    return {
        'question': q,
        'research': research,
        'final': rewrite,
        'models': {
            'researcher': PRIMARY_MODEL_ID,
            'editor': EDITOR_MODEL_ID
        }
    }

# Execute sample pipeline
print("="*80)
result = pipeline('Explain why edge AI matters for compliance and latency.')
print("="*80)
print("\n[FINAL OUTPUT]")
print(result['final'])
print("\n[METADATA]")
print(f"Models used: {result['models']}")
result

[Pipeline] Question: Explain why edge AI matters for compliance and latency.

[Stage 1: Research]
Output: - **Data Sovereignty**: Edge AI allows data to be processed locally, which can help organizations comply with regional data protection regulations by keeping sensitive information within the borders o...

[Stage 2: Editorial Refinement]


### Paliwanag: Pagpapatakbo ng Pipeline at Resulta
Isinasagawa ang multi-agent pipeline sa isang tanong na may tema tungkol sa pagsunod at latency upang ipakita:
- Multi-stage na pagbabago ng impormasyon
- Espesyalisasyon at kolaborasyon ng mga ahente
- Pagpapabuti ng kalidad ng output sa pamamagitan ng refinement
- Traceability (napananatili ang parehong intermediate at final na mga output)

**Struktura ng Resulta:**
- `question` - Orihinal na tanong ng user
- `research` - Hilaw na output ng pananaliksik (mga factual na puntos)
- `final` - Pinong executive summary
- `models` - Aling mga modelo ang ginamit sa bawat yugto

**Mga Ideya para sa Pagpapalawak:**
1. Magdagdag ng Critic agent para sa pagsusuri ng kalidad
2. Magpatupad ng parallel na mga research agent para sa iba't ibang aspeto
3. Magdagdag ng Verifier agent para sa pag-check ng katotohanan
4. Gumamit ng iba't ibang modelo para sa iba't ibang antas ng komplikasyon
5. Magpatupad ng feedback loops para sa iterative na pagpapabuti


### Advanced: Pasadyang Konfigurasyon ng Ahente

Subukan ang pagpapasadya ng kilos ng ahente sa pamamagitan ng pagbabago ng mga environment variable bago patakbuhin ang initialization cell:

**Mga Available na Modelo:**
- Gamitin ang `foundry model ls` sa terminal upang makita ang lahat ng available na modelo
- Mga Halimbawa: phi-4-mini, phi-3.5-mini, qwen2.5-7b, llama-3.2-3b, atbp.


In [ ]:
# Example: Use different models for different agents
# Uncomment and modify as needed:

# import os
# os.environ['AGENT_MODEL_PRIMARY'] = 'phi-4-mini'      # Fast, good for research
# os.environ['AGENT_MODEL_EDITOR'] = 'qwen2.5-7b'       # Higher quality for editing

# Then restart the kernel and re-run all cells

# Test with different questions
test_questions = [
    "What are 3 key benefits of using small language models?",
    "How does RAG improve AI accuracy?",
    "Why is local inference important for privacy?"
]

print("Testing pipeline with multiple questions:\n")
for i, q in enumerate(test_questions, 1):
    print(f"\n{'='*80}")
    print(f"Question {i}: {q}")
    print('='*80)
    r = pipeline(q, verbose=False)
    print(f"\n[FINAL]: {r['final'][:300]}...")
    print(f"[Models]: Researcher={r['models']['researcher']}, Editor={r['models']['editor']}")


Testing pipeline with multiple questions:


Question 1: What are 3 key benefits of using small language models?

[FINAL]: <|channel|>analysis<|message|>The user wants a rewrite of the entire block of text. The rewrite should be professional, include a one-sentence executive summary first, improve clarity, keep bullet structure if present. The user has provided a large amount of text. The user wants a rewrite of that te...
[Models]: Researcher=Phi-4-mini-instruct-cuda-gpu:4, Editor=gpt-oss-20b-cuda-gpu:1

Question 2: How does RAG improve AI accuracy?

[FINAL]: <|channel|>final<|message|>**RAG (Retrieval‑Augmented Generation) empowers AI to produce highly accurate, contextually relevant responses by combining a retrieval system with a large language model (LLM).**<|return|>...
[Models]: Researcher=Phi-4-mini-instruct-cuda-gpu:4, Editor=gpt-oss-20b-cuda-gpu:1

Question 3: Why is local inference important for privacy?

[FINAL]: <|channel|>final<|message|>**Local inference—processing data d


---

**Paunawa**:  
Ang dokumentong ito ay isinalin gamit ang AI translation service na [Co-op Translator](https://github.com/Azure/co-op-translator). Bagama't sinisikap naming maging tumpak, mangyaring tandaan na ang mga awtomatikong pagsasalin ay maaaring maglaman ng mga pagkakamali o hindi pagkakatugma. Ang orihinal na dokumento sa kanyang katutubong wika ang dapat ituring na opisyal na sanggunian. Para sa mahalagang impormasyon, inirerekomenda ang propesyonal na pagsasalin ng tao. Hindi kami mananagot sa anumang hindi pagkakaunawaan o maling interpretasyon na dulot ng paggamit ng pagsasaling ito.
